In [ ]:
## --- parameter ---
CONTEST = "W395" # Contest name
CONTEST_URL = "https://leetcode.com/contest/weekly-contest-395/" # Contest URL
CONTEST_API_URL = "https://leetcode.com/contest/api/ranking/weekly-contest-395/" # API URL
PAGE = 125 # how many page gonna fetch

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.9',
    'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
}

In [ ]:
import csv
import json
import os
import sys
from tqdm import tqdm
import urllib.request

user = []
Q3 = []
Q4 = []
Q = []

## get question_id
for _ in range(10):
    try:
        req = urllib.request.Request(CONTEST_API_URL, headers=headers)
        r = urllib.request.urlopen(req).read()
        response = json.loads(r)
    except:
        pass
    else:
        break
else:
    print("http request error:", CONTEST_API_URL)
    sys.exit()

for question in response["questions"]:
    Q.append(str(question["question_id"]))
    print(question["question_id"], question["credit"], question["title"])

## get ranking page
for pagination in tqdm(range(1, PAGE+1)):
    for _ in range(10):
        try:
            req = urllib.request.Request(CONTEST_API_URL+"?pagination=" + str(pagination) +"&region=global", headers=headers)
            r = urllib.request.urlopen(req).read()
            response = json.loads(r)
        except:
            pass
        else:
            break
    else:
        print("http request error:", CONTEST_API_URL+"?pagination=" + str(pagination) +"&region=global")
        sys.exit()

    for i in range(25):
        user.append(response["total_rank"][i]["user_slug"])
        if Q[2] not in response["submissions"][i]:
            Q3.append("")
        else:   
            Q3.append(response["submissions"][i][Q[2]]["submission_id"])
        if Q[3] not in response["submissions"][i]:
            Q4.append("")
        else:
            Q4.append(response["submissions"][i][Q[3]]["submission_id"])

In [ ]:
cheater = []
DQed = []
for filename in os.listdir("history/"+CONTEST):
    f_path = os.path.join("history/"+CONTEST, filename)
    if os.path.isfile(f_path):
        QUESTION = filename[0:2]
        
        with open(f_path, "r") as f:
            rows = list(csv.reader(f))
        
        DQ = 0
        with open(filename+".md", "w") as md:
            print("| rank | user | "+QUESTION+"   |\n| ---- | ---- | ---- |", file=md)
            
            with open(f_path, "w") as f:
                for i in rows:
                    try:        
                        if QUESTION == "Q4":
                            idx = Q4.index(int(i[2]))
                        elif QUESTION == "Q3":
                            idx = Q3.index(int(i[2]))
                        else:
                            sys.exit()
                    except:
                        try:
                            idx = user.index(i[1])
                        except:
                            idx = -1
                    
                    
                    if idx != -1:
                        cheater.append(user[idx])
                        
                        print(str(idx+1) + "," + user[idx] + "," + i[2], file=f)
                        print("|", idx+1, "| ["+user[idx]+"](https://leetcode.com/u/"+user[idx]+") |", CONTEST_URL+"submissions/detail/"+i[2], "|", file=md)
                    else:
                        cheater.append(i[1])
                        DQed.append(i[1])
                        
                        DQ += 1
                        print("DQ," + i[1] + "," + i[2], file=f)
                        print("| DQ | ["+i[1]+"](https://leetcode.com/u/"+i[1]+") |", CONTEST_URL+"submissions/detail/"+i[2], "|", file=md)
        
        print(filename, "DQ:", DQ, '/', len(rows))

print("DQ:", len(set(DQed)), '/', len(set(cheater)))